In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.backends.cudnn.enabled)
print(torch.cuda.is_available(), device)
block_size = 8
batch_size = 4
max_iters = 100000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 10000
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

True
True cuda


In [ ]:
with open("Text/wizardofoz.txt", 'r', encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [ ]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [ ]:
n = int(.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i+1:i + block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch("train")
print("inputs:")
print(x)
print("targets: ")
print(y)

inputs:
tensor([[73, 61, 58, 71,  0, 62, 66, 69],
        [ 1, 61, 54, 75, 58,  1, 55, 58],
        [76, 61, 58, 67,  1, 73, 61, 58],
        [57, 62, 65, 78,  1, 54, 73,  1]], device='cuda:0')
targets: 
tensor([[61, 58, 71,  0, 62, 66, 69, 68],
        [61, 54, 75, 58,  1, 55, 58, 58],
        [61, 58, 67,  1, 73, 61, 58, 78],
        [62, 65, 78,  1, 54, 73,  1, 73]], device='cuda:0')


In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    print("when input is", x[:t+1], "target is", y[t])

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList()
        self.head_size = nn.MudleList([Head(head_size) for _ in range(num_heads)])
        self.pro = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
      out = torch.cat([h(x) for h in self.heads], dim=-1)
      out = self.droupout(self.proj(out))
      return out
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        y = self.sa(x)
        x = self.ln(x + y)
        y = self.ffwd(x)
        x = self.ln2(x +y )
        return x

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=nhead) for _ in range(n_layer)])
        #creates n_layer of decoder blocks ^
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    def forward(self, index, targets=None):
        B, T = index.shape
        #idx and targets are both (B, T) tensors of integers
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            if _ == 0:
                print(logits)
                print(loss)
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(context)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

tensor([[0]])
tensor([[ 7.8373e-01, -2.8364e+00,  4.2398e-02,  3.9636e-01, -7.1273e-01,
          1.4471e+00,  3.1153e-01,  1.0191e+00,  1.7920e+00, -4.7153e-01,
          2.3163e-01,  1.7303e+00, -3.6936e-01,  5.3360e-01, -1.5936e+00,
          1.5642e+00, -3.4820e-01,  1.1174e-01,  2.2142e-01, -4.8586e-01,
          4.9699e-01, -3.7214e-01,  1.2762e+00,  2.1241e+00, -1.7587e-01,
          8.8144e-01,  9.7419e-01, -7.8362e-01, -3.9627e-02,  1.1407e+00,
         -1.0389e+00, -3.9734e-02,  4.1541e-04,  4.3913e-01, -8.7721e-01,
          2.8870e-01,  1.4294e+00,  7.0931e-01, -1.6924e+00, -1.6432e+00,
          5.1453e-01,  5.1636e-01,  1.3501e+00, -1.7962e+00, -5.2437e-02,
          1.3237e+00, -4.6333e-01, -1.6825e+00,  1.7451e+00, -1.0982e+00,
          2.3132e+00, -1.1442e-01,  9.0815e-01, -2.3807e+00,  6.3099e-01,
         -4.7849e-01,  4.2807e-02, -1.4317e+00,  7.6534e-01,  4.7763e-01,
         -9.5179e-01, -6.1734e-01, -3.9940e-01,  1.5688e-01, -6.6271e-01,
         -3.4274e-01,  7

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    #if iter % eval_iters == 0:
        #losses = estimate_loss()
        #print(f"step: {iter} train loss: {losses["train"]} val loss {losses["val"]}")

    xb, yb = get_batch("train")
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.2805185317993164


In [ ]:
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

tensor([[ 2.4179e+00, -9.5441e-01, -1.2724e+01,  1.9069e+00, -1.2725e+01,
         -1.2722e+01, -1.2724e+01, -1.2723e+01, -1.2721e+01, -1.2725e+01,
         -1.2724e+01, -1.2721e+01, -1.2724e+01, -1.2723e+01, -1.2726e+01,
         -1.2722e+01, -1.2724e+01, -1.2724e+01, -1.2724e+01, -1.2725e+01,
         -1.2723e+01, -1.2724e+01, -1.2722e+01, -1.2721e+01, -1.2724e+01,
         -1.1944e+00, -1.1990e+00, -1.4076e+00, -1.0405e+00, -1.8594e+00,
         -2.5106e+00, -2.6229e+00, -1.4731e+00, -1.0993e+00, -1.4646e+00,
         -6.0136e+00, -3.1903e+00, -3.1464e+00, -1.9552e+00, -1.8224e+00,
         -2.1634e+00, -1.2723e+01, -7.0102e+00, -9.5782e-01,  4.1367e-01,
         -5.9449e+00, -3.4331e+00, -1.2502e+00, -1.2721e+01, -1.2725e+01,
         -1.7813e+00, -1.0236e+00, -1.2723e+01, -3.3835e+00,  6.4830e-01,
          3.9218e-02,  1.1205e-02, -7.5618e-01, -5.4659e-01, -2.4282e-01,
         -8.7273e-01, -1.2857e-01, -3.5768e-01, -2.9220e+00, -2.3416e+00,
         -5.4720e-01, -5.9060e-01, -1.